# Librerías

In [43]:
import itertools
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import offsetbox
from sklearn import (manifold, datasets, decomposition, ensemble,
                     discriminant_analysis, random_projection)#Prueba evaluación y entrenamiento un solo individuo. 
#Vectores completos
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
import random

# Databases

In [44]:
#Reading all the cvs
#Nss learning
nsl_training=pd.read_csv('./Final_Datasets/MINMAX_FD_NSL_Tra.csv')
#Nsl testing
nsl_testing=pd.read_csv('./Final_Datasets/MINMAX_FD_NSL_Test.csv')

# Labels

In [45]:
#NSL TESTING LABELS

#Index binary (2 clases)
L_nsl_testing_indexbinary=pd.read_csv("./Final_Datasets/L_nsl_testing_indexbinary.csv")
#Index by Group (5 clases)
L_nsl_testing_indexbygroup=pd.read_csv("./Final_Datasets/L_nsl_testing_indexbygroup.csv")
#Index tipe of attack (42? clases)
L_nsl_testing_indextipeofattack=pd.read_csv("./Final_Datasets/L_nsl_testing_indextipeofattack.csv")


#nsl TRAINING LABELS
# 2 clases
L_nsl_training_indexbinary=pd.read_csv("./Final_Datasets/L_nsl_training_indexbinary.csv")
#Index by Group (5 clases)
L_nsl_training_indexbygroup=pd.read_csv("./Final_Datasets/L_nsl_training_indexbygroup.csv")
#Index tipe of attack (42? clases)
L_nsl_training_indextipeofattack=pd.read_csv("./Final_Datasets/L_nsl_training_indextipeofattack.csv")

# Conversión de Pandas->Numpy (array) not Matrix

In [46]:
nsl_training=nsl_training.as_matrix(columns=None)
nsl_testing=nsl_testing.as_matrix(columns=None)

# Lectura del modelo

In [47]:
from sklearn.ensemble import RandomForestClassifier

modelo=RandomForestClassifier(criterion='entropy',n_jobs=2, 
                              bootstrap=True, max_depth=10, max_features='sqrt', min_samples_leaf=2, 
                              min_samples_split=100, n_estimators=8)


# Creacion de las funciones a utilizar

# funcion muta vector binario corregida

In [48]:
def muta_vector_binario(dimensiones_temporales,M=20):
    """Dimensiones temporales: vector binario que contiene las dimensiones actuales de la ultima
        iteración probada.
        M=cantidad total de dimensiones a mutar. Si no se otorga un número
        M es igual a la máxima cantidad de dimensiones en NSL-KDD"""
    
    print("Valor de M (dimensiones a mutar): %s" %M)
    vector_mutado=np.array(dimensiones_temporales)
    #Genera números aleatorios entre 0 y M
    #Dichos numeros aleatorios seran los indices a mutar de (vector_mutado)
    indice_general=list(range(0,40))
    indice=sorted(random.sample(indice_general,M))
    
   
    #Aplica la operación negación binaria a los valores del indice
    for numero_indice in indice:
        if vector_mutado[numero_indice]==0:
            vector_mutado[numero_indice]=1
        else:
            vector_mutado[numero_indice]=0
    #regresa el vector mutado
    print("indice de vectores a mutar: ")
    print(indice)
    print("Solución actual:")
    print(dimensiones_temporales)
    print("Solución mutada:")
    print(vector_mutado)
    print("\n")
    return vector_mutado
    

In [49]:
def obtencion_parametros_binarios(vector_actual, M=40):
    """M=máximo número de dimensiones a mutar
    vector_actual= vector binario, relacionado con la ultima iteracion
    puede o no ser el mejor.
    Regresa los zeros y unos actuales,
    el ultimo vector binario,
    los dataset filtrados"""

    #Agregar la función para calcular M

    
    #Muta el ultimo vector
    dimensiones_temporales=muta_vector_binario(vector_actual,M)

    #se filtra la base de datos con las dimensiones de dimensiones mejor
    #SE hacen cero las dimensiones a omitir
    #dimensiones_temporales contiene la mejor iteración, entonces es la dimensión activa
    training=nsl_training*dimensiones_temporales
    testing=nsl_testing*dimensiones_temporales

    #Se buscan esos ceros en el indice (dimensiones suspendidas)
    zeros=np.where(dimensiones_temporales == 0)

    #se obtienen los indices del los unos (dimensiones activas)
    unos=np.where(dimensiones_temporales==1)

    #se eliminan esas dimensiones
    training=np.delete(training,zeros,axis=1)
    testing=np.delete(testing,zeros,axis=1)

    
    return [dimensiones_temporales,zeros,unos,training,testing]

# Funcion para calcular M dependiendo de las iteraciones

In [50]:
def M_dimensiones_a_mutar(Perror,i_actual,i_max,M_Max=40):
    iteraciones=1-(i_actual/i_max)
    M=M_Max*np.amin([iteraciones,Perror])
    M=np.array(np.round_(M))
    return M

In [54]:
def M_dimensiones_a_mutar_2(i_actual,i_max,M_Max=40):
    iteraciones=1-(i_actual/i_max)
    M=M_Max*iteraciones
    M=np.array(np.round_(M))
    return M

# Declaración del modelo

In [51]:
from sklearn.ensemble import RandomForestClassifier

modelo=RandomForestClassifier(criterion='entropy',n_jobs=2, 
                              bootstrap=True, max_depth=10, max_features='sqrt', min_samples_leaf=2, 
                              min_samples_split=100, n_estimators=8)


# Función de rendimiento

In [52]:
def funcion_rendimiento(Perror_actual,unos,N=40,alfa=.3):
    cardinalidad=np.array(unos).shape[1]
    rendimiento=alfa*Perror_actual+(1-alfa)*(cardinalidad/N)
    return rendimiento

# Inicio del algoritmo RMHC

In [56]:
i=1
M=38
iteraciones_maximas=15
#la declaracion tiene que ser 40! cómo puede tener menos dimensinoes???!!!
mejor_solucion=np.random.randint(2, size=40)
#almacena los score de todas las iteraciones
registro_score_iteracion=[]
#almacena la probabilidad de error de todas las iteraciones
registro_perror_iteracion=[]
#almacena el vector binario de todas las iteraciones
registro_dimensiones_iteracion=[]
#registro de la probabilidad de error de los valores aceptados
registro_perror_valores_aceptados=[]
#almacena unicamente el score de los vectores binarios aceptados
registro_score_valores_aceptados=[]
#almacena el vector binario de los valores aceptados (dimensiones_activas)
registro_dimensiones_activas_valores_aceptados=[]
#registro de las dimensiones activas por iteracion
registro_dimensiones_activas_iteracion=[]
#registro de la cantidad de dimensiones activas por iteracion
registro_cantidad_dimensiones_activas_iteracion=[]
#registro del rendimiento por iteracion
registro_rendimiento_iteracion=[]
#registro del rendimiento de los valore aceptados
registro_rendimiento_aceptados=[]



print("solucion_inicial")
print(mejor_solucion)
print("\n")

dimensiones_temporales=np.array(mejor_solucion)

while (i<iteraciones_maximas):
    print("\n\n Iteracion: %s \n\n" %(i))
    
    [dimensiones_temporales,zeros,unos,training,testing]=obtencion_parametros_binarios(dimensiones_temporales,M)    
    #Se evaluan los resultados
    #se entrena el modelo
    modelo.fit(training,L_nsl_training_indexbinary.values.ravel())
    #se obtienen las predicciones
    predicciones=modelo.predict(testing)

    #se obtiene el accuracy score
    score_actual=accuracy_score(L_nsl_testing_indexbinary,predicciones)
    #se obtiene la probabilidad de error
    perror_actual=1-score_actual
    #se obtiene el rendimiento de dicha solucion
    rendimiento_actual=funcion_rendimiento(perror_actual,unos,N=40,alfa=.3)
    
    
    
    
    
    #se declara un vector que contiene todos los score de todas las iteraciones

    #Se almacenan los resultados
    registro_score_iteracion.append(score_actual)
    registro_dimensiones_iteracion.append(dimensiones_temporales)
    registro_perror_iteracion.append(perror_actual)
    registro_dimensiones_activas_iteracion.append(unos)
    registro_cantidad_dimensiones_activas_iteracion.append(np.array(unos).shape[1])
    registro_rendimiento_iteracion.append(rendimiento_actual)
    
    #Se imprimen los resultados
    print("Indice de las Dimensiones activas: ")
    print(unos)
    print("Total de dimensiones activas: %s" % np.array(unos).shape[1])
    print("Porcentaje de exactitud: %s (accuracy score)" %accuracy_score(L_nsl_testing_indexbinary,predicciones))
    print("Probabilidad de error (Perror): %s" % (1-accuracy_score(L_nsl_testing_indexbinary,predicciones)))
    print("Rendimiento (RE): %s" %rendimiento_actual)
    
    #M se calcula al final
    
    #M=M_dimensiones_a_mutar_2(i,iteraciones_maximas,M_Max=40)
    
    M=M_dimensiones_a_mutar(perror_actual,i,iteraciones_maximas,M_Max=40)
    
    
    #M debe ser entero o da un error al elegir el sampleo aleatorio en la seleccion de dimensiones
    #problema porque M es flotante
    M=int(M)
   
    
    i=i+1
    

solucion_inicial
[1 0 0 0 1 1 0 1 0 1 0 1 0 0 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 0 0 0 1 0 0 0 1
 1 1 0]




 Iteracion: 1 


Valor de M (dimensiones a mutar): 38
indice de vectores a mutar: 
[0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 34, 35, 36, 37, 38, 39]
Solución actual:
[1 0 0 0 1 1 0 1 0 1 0 1 0 0 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 0 0 0 1 0 0 0 1
 1 1 0]
Solución mutada:
[0 0 1 1 0 0 1 0 1 0 1 0 1 1 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1 1 1 0 0 1 1 0
 0 0 1]


Indice de las Dimensiones activas: 
(array([ 2,  3,  6,  8, 10, 12, 13, 17, 20, 29, 30, 31, 34, 35, 39]),)
Total de dimensiones activas: 15
Porcentaje de exactitud: 0.769329725414 (accuracy score)
Probabilidad de error (Perror): 0.230670274586
Rendimiento (RE): 0.331701082376


 Iteracion: 2 


Valor de M (dimensiones a mutar): 9
indice de vectores a mutar: 
[1, 11, 16, 18, 22, 25, 29, 33, 36]
Solución actual:
[0 0 1 1 0 0 1 0 1 0 1 0 1 1 0 0 0 1 0 0 1 0 0 0 0 0 

Indice de las Dimensiones activas: 
(array([ 2,  3,  4,  7,  8, 15, 16, 17, 20, 21, 23, 24, 25, 27, 30, 31, 33,
       34, 36, 37, 39]),)
Total de dimensiones activas: 21
Porcentaje de exactitud: 0.765292995608 (accuracy score)
Probabilidad de error (Perror): 0.234707004392
Rendimiento (RE): 0.437912101317


In [13]:
dimensiones_temporales.shape

(40,)

In [21]:
len(unos)

1

In [38]:
np.array(unos).shape[1]

17